In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ============================================
# Benchmark meta-llama/Llama-3.2-3B-Instruct on GSM-Symbolic with HF token
# ============================================

!pip install -q transformers torch tqdm accelerate bitsandbytes huggingface_hub pandas

import os
import json
import time
import re
import random
import numpy as np
from tqdm import tqdm
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from huggingface_hub import login

# ---------- Setup ----------
os.makedirs("/kaggle/working/results", exist_ok=True)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# ---------- Hugging Face Token ----------
HF_TOKEN = os.environ.get("HUGGINGFACE_TOKEN", "")
login(token=HF_TOKEN)

# ---------- Utility functions ----------
def normalize_answer(ans: str) -> str:
    return ans.strip().lower().replace(",", "")

def is_answer_correct(pred, gold):
    """Compare numeric answers, ignoring formatting and symbols like % or commas."""
    pred_nums = re.findall(r"[-+]?\d*\.?\d+", pred)
    gold_nums = re.findall(r"[-+]?\d*\.?\d+", gold)

    if not pred_nums or not gold_nums:
        return pred.strip().lower() == gold.strip().lower()

    try:
        pred_val = float(pred_nums[-1])
        gold_val = float(gold_nums[-1])
        return abs(pred_val - gold_val) < 1e-3
    except ValueError:
        return False

# ---------- Benchmark class ----------
class LlamaBenchmark:
    def __init__(self, model_name="meta-llama/Llama-3.2-3B-Instruct",
                 device=None, max_new_tokens=512, temperature=0.0, hf_token=None):
        print(f"🔹 Loading model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            use_auth_token=hf_token
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
            use_auth_token=hf_token
        )
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature

    def make_prompt(self, question):
        return (
            "You are a math reasoning assistant. "
            "Solve the problem step by step and end with a numeric answer'Answer: {number}'.\n\n"
            f"Problem: {question}\n\nSolution:\n"
        )

    def extract_answer(self, text):
        """Extract final numeric or text answer robustly."""
        if re.search(r"(final answer|answer is)[:=]", text, re.I):
            ans = re.split(r"(final answer|answer is)[:=]", text, flags=re.I)[-1].strip()
            return ans.split("\n")[0]
        if "Answer:" in text:
            ans = text.split("Answer:")[-1].strip()
            return ans.split("\n")[0]
        nums = re.findall(r"[-+]?\d*\.?\d+", text)
        return nums[-1].strip() if nums else text.strip().split("\n")[-1]

    @torch.inference_mode()
    def run_one(self, question):
        prompt = self.make_prompt(question)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        start = time.time()

        # --- Deterministic first pass ---
        gen_config = GenerationConfig(
            max_new_tokens=self.max_new_tokens,
            do_sample=False,
            pad_token_id=self.tokenizer.eos_token_id
        )

        output = self.model.generate(
            **inputs,
            generation_config=gen_config
        )
        latency = time.time() - start
        decoded = self.tokenizer.decode(output[0], skip_special_tokens=True)
        pred = self.extract_answer(decoded)

        # --- Retry with sampling if output is junk ---
        if len(pred) < 2 or pred in {"'", '"', ".", ":", ";"}:
            gen_config_retry = GenerationConfig(
                max_new_tokens=self.max_new_tokens,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

            output = self.model.generate(
                **inputs,
                generation_config=gen_config_retry
            )
            decoded = self.tokenizer.decode(output[0], skip_special_tokens=True)
            pred = self.extract_answer(decoded)

        return decoded, pred, latency

    def benchmark(self, dataset, save_path, q_key="question", a_key="answer", num_examples=None, checkpoint_interval=50):
        correct, latencies = 0, []
        total_examples = num_examples or len(dataset)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Resume support
        processed = set()
        if os.path.exists(save_path):
            with open(save_path, "r", encoding="utf-8") as f:
                for line in f:
                    try:
                        data = json.loads(line)
                        processed.add(data["index"])
                        if data.get("correct"):
                            correct += 1
                        latencies.append(data.get("latency", 0))
                    except:
                        continue
            print(f"🔄 Resuming from {len(processed)} completed examples...")

        with open(save_path, "a", encoding="utf-8") as f:
            for i, ex in enumerate(tqdm(dataset, desc=f"Running {save_path}")):
                if i in processed:
                    continue
                if num_examples and i >= num_examples:
                    break

                q, gold = ex[q_key], str(ex[a_key])
                decoded, pred, t = self.run_one(q)
                ok = is_answer_correct(pred, gold)

                result = {
                    "index": i,
                    "question": q,
                    "gold_answer": gold,
                    "prediction": pred,
                    "correct": ok,
                    "latency": t,
                }

                f.write(json.dumps(result) + "\n")
                f.flush()
                os.fsync(f.fileno())

                if i % checkpoint_interval == 0:
                    print(json.dumps(result, indent=2))

                correct += ok
                latencies.append(t)

                # Save interim summaries
                if i % 500 == 0 and i > 0:
                    interim = {"processed": i, "correct": correct, "total": len(latencies)}
                    with open(save_path.replace(".jsonl", "_summary.json"), "w") as s:
                        json.dump(interim, s, indent=2)

        total = len(latencies)
        acc = round(100 * correct / total, 2) if total else 0
        avg_t = round(sum(latencies) / total, 3) if total else 0
        print(f"\n✅ {save_path}: {correct}/{total} correct → {acc}% | avg latency {avg_t}s")

        return {"total": total, "correct": correct, "accuracy": acc, "avg_latency": avg_t}

# ---------- Load GSM-Symbolic dataset using pandas ----------
hf_path = "hf://datasets/apple/GSM-Symbolic/main/test.jsonl"
gsm_df = pd.read_json(hf_path, lines=True)
print("Columns:", gsm_df.columns)

# Convert to list of dicts for benchmark
gsm_symbolic = gsm_df.to_dict(orient="records")

# ---------- Run benchmark ----------
bench = LlamaBenchmark("meta-llama/Llama-3.2-3B-Instruct", hf_token=HF_TOKEN)

gsm_metrics = bench.benchmark(
    gsm_symbolic,
    "/kaggle/working/results/llama3_gsm_symbolic.jsonl",
    q_key="question",
    a_key="answer",
    checkpoint_interval=200
)

# ---------- Save summary ----------
summary = {"gsm_symbolic": gsm_metrics}
with open("/kaggle/working/results/summary_llama3.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n📂 All results stored in /kaggle/working/results/")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Columns: Index(['id', 'instance', 'question', 'answer', 'original_id',
       'original_question', 'original_answer', 'canary'],
      dtype='object')
🔹 Loading model: meta-llama/Llama-3.2-3B-Instruct


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔄 Resuming from 826 completed examples...


...

{
  "index": 4600,
  "question": "Jin slew 150 goblins with his mighty sword, while DeShawn, using a spear, slew five-sixths as many goblins as Jin. Using a rusty iron axe, Gabriel slew two times as many goblins as DeShawn. But Oscar, having forgotten his sword at home, slew three-sixths as many goblins as Gabriel using a nail file. How many goblins has Oscar slain?",
  "gold_answer": "DeShawn slew 5/6 as many goblins as Jin, or 150*5/6=125 goblins.\nGabriel slew 2 as many goblins as DeShawn, or 2*125=250 goblins.\nOscar slew 1/2 as many goblins as Gabriel, or 250*1/2=125 goblins.\n#### 125",
  "prediction": "125",
  "correct": true,
  "latency": 6.87434983253479
}


Running /kaggle/working/results/llama3_gsm_symbolic.jsonl:  96%|█████████▌| 4801/5000 [6:36:01<22:31,  6.79s/it]  

{
  "index": 4800,
  "question": "Mia's aunt said that she had \u20ba300 budgeted for her birthday party. She wants to make sure she and her friends all get to play one round of laser tag, have \u20ba5 in pinball tokens, and get to ride the roller coaster seven times. A round of laser tag is \u20ba10. The roller coaster cost \u20ba5 a ride. How many friends can she invite?",
  "gold_answer": "The roller coaster will cost \u20ba35 per person because 5 x 7 = 35\nEach person costs \u20ba50 because 5 + 10 + 35 = 50\n6 total people can attend because 300 / 50 = 6\nShe can invite 5 friends because 6 - 1 =5\n#### 5",
  "prediction": "25",
  "correct": false,
  "latency": 3.5877671241760254
}


Running /kaggle/working/results/llama3_gsm_symbolic.jsonl: 100%|██████████| 5000/5000 [7:00:14<00:00,  5.04s/it]


✅ /kaggle/working/results/llama3_gsm_symbolic.jsonl: 3493/5000 correct → 69.86% | avg latency 5.703s

📂 All results stored in /kaggle/working/results/


In [4]:
!zip -r results.zip /kaggle/working/
from IPython.display import FileLink
FileLink('results.zip')

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/results/ (stored 0%)
  adding: kaggle/working/results/summary_llama3.json (deflated 21%)
  adding: kaggle/working/results/llama3_gsm_symbolic_summary.json (deflated 15%)
  adding: kaggle/working/results/llama3_gsm_symbolic.jsonl (deflated 88%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/kaggle/working/results.zip